In [ ]:
# | default_exp _components.aiokafka_producer_manager

In [ ]:
# | export

import asyncio
from contextlib import asynccontextmanager
from typing import *
import time
import logging

import anyio
from aiokafka import AIOKafkaProducer
from anyio.streams.memory import MemoryObjectReceiveStream, MemoryObjectSendStream

from fastkafka._components.logger import get_logger, cached_log

In [ ]:
import unittest.mock
from contextlib import contextmanager
from os import environ

import nest_asyncio

from fastkafka._components.logger import supress_timestamps
from fastkafka.testing import ApacheKafkaBroker

In [ ]:
# | notest

# allows async calls in notebooks
nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=1)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


@asynccontextmanager
async def _aiokafka_producer_manager(  # type: ignore # Argument 1 to "_aiokafka_producer_manager" becomes "Any" due to an unfollowed import  [no-any-unimported]
    producer: AIOKafkaProducer,
    *,
    max_buffer_size: int = 1_000_000,
) -> AsyncGenerator[MemoryObjectSendStream[Any], None]:
    """Write docs

    Todo: add batch size if needed

    """

    logger.info("_aiokafka_producer_manager(): Starting...")

    async def send_message(receive_stream: MemoryObjectReceiveStream) -> Any:
        async with receive_stream:
            async for topic, msg, key in receive_stream:
                fut = await producer.send(topic, msg, key=key)
                
                def release_callbalck(fut):
                    pass
                
                fut.add_done_callback(release_callbalck)

    send_stream, receive_stream = anyio.create_memory_object_stream(
        max_buffer_size=max_buffer_size
    )

    logger.info("_aiokafka_producer_manager(): Starting send_stream")
    asyncio.create_task(send_message(receive_stream))
    async with send_stream:
        yield send_stream
        logger.info("_aiokafka_producer_manager(): Exiting send_stream")

    logger.info("_aiokafka_producer_manager(): Finished.")

In [ ]:
@contextmanager
def mock_AIOKafkaProducer_send():
    with unittest.mock.patch("__main__.AIOKafkaProducer.send") as mock:

        async def _f():
            pass

        mock.return_value = asyncio.create_task(_f())

        yield mock

In [ ]:
num_msgs = 15
topic = "topic"
msg = b"msg"
key = None
msgs = [(topic, msg, key) for _ in range(num_msgs)]
calls = [unittest.mock.call(topic, msg, key=key) for _ in range(num_msgs)]

with mock_AIOKafkaProducer_send() as send_mock:
    producer = AIOKafkaProducer()
    async with _aiokafka_producer_manager(producer) as send_stream:
        for msg in msgs:
            send_stream.send_nowait(msg)

        await asyncio.sleep(10)

        await producer.stop()

    send_mock.assert_has_calls(calls)

[INFO] __main__: _aiokafka_producer_manager(): Starting...
[INFO] __main__: _aiokafka_producer_manager(): Starting send_stream
[DEBUG] aiokafka.producer.producer: The Kafka producer has closed.
[INFO] __main__: _aiokafka_producer_manager(): Exiting send_stream
[INFO] __main__: _aiokafka_producer_manager(): Finished.


In [ ]:
# | export


class AIOKafkaProducerManager:
    def __init__(self, producer: AIOKafkaProducer, *, max_buffer_size: int = 1_000):  # type: ignore
        self.producer = producer
        self.max_buffer_size = max_buffer_size
        self.shutting_down = False

    async def start(self) -> None:
        logger.info("AIOKafkaProducerManager.start(): Entering...")
        await self.producer.start()
        self.producer_manager_generator = _aiokafka_producer_manager(
            self.producer, max_buffer_size=self.max_buffer_size
        )
        self.send_stream = await self.producer_manager_generator.__aenter__()
        self.shutting_down = False
        logger.info("AIOKafkaProducerManager.start(): Finished.")

    async def stop(self) -> None:
        # todo: try to flush messages before you exit
        logger.info("AIOKafkaProducerManager.stop(): Entering...")
        self.shutting_down = True
        await self.producer_manager_generator.__aexit__(None, None, None)
        logger.info("AIOKafkaProducerManager.stop(): Stoping producer...")
        await self.producer.flush()
        await self.producer.stop()
        logger.info("AIOKafkaProducerManager.stop(): Finished")

    async def _send_with_throttle(self, *, topic, msg, key, stream):
        while not self.shutting_down:
            try:
                stream.send_nowait(data)
                break
            except anyio.WouldBlock:
                cached_log(
                    logger,
                    f"Send stream full and blocking for {topic=}, throttling...",
                    level=logging.WARNING,
                )
                await asyncio.sleep(0.01)

    def send(self, topic: str, msg: bytes, key: Optional[bytes] = None) -> None:
        if not self.shutting_down:
            asyncio.create_task(
                self._send_with_throttle(topic=topic, msg=msg, key=key, stream=self.send_stream)
            )

In [ ]:
async with ApacheKafkaBroker() as bootstrap_server:
    producer = AIOKafkaProducer(bootstrap_servers=bootstrap_server)
    manager = AIOKafkaProducerManager(producer)
    await manager.start()
    manager.send("topic", b"msg")
    await manager.stop()
    logger.info("Stopped")

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
[INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
[INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] __main__: AIOKafkaProducerManager.start(): Entering...
[DEBUG] aiokafka.producer.producer: Starting the Kafka producer
[DEBUG] aiokafka: Attempting to bootstrap via node at 127.0.0.1:9092
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=127.0.0.1 port=9092> Request 1: MetadataRequest_v0(topics=[])
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=127.0.0.1 port=9092> Response 1: MetadataResponse_v0(brokers=[(node_id=0, host='tvrtko-fastkafka-